<h1><center>Statistical Comparison of ML Models</center></h1>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Data sample

## MAGIC – Major Atmospheric Gamma Imaging Cherenkov Telescope

<center><img src="img/magic1.jpg" width="1000"></center>

Source: https://magic.mpp.mpg.de/

The data are MC generated (see below) to simulate registration of high energy gamma particles in a ground-based atmospheric Cherenkov gamma telescope using the imaging technique. Cherenkov gamma telescope observes high energy gamma rays, taking advantage of the radiation emitted by charged particles produced inside the electromagnetic showers initiated by the gammas, and developing in the atmosphere.

<center><img src="img/shower.jpg" width="500"></center>

This Cherenkov radiation (of visible to UV wavelengths) leaks through the atmosphere and gets recorded in the detector, allowing reconstruction of the shower parameters. The available information consists of pulses left by the incoming Cherenkov photons on the photomultiplier tubes, arranged in a plane, the camera. Depending on the energy of the primary gamma, a total of few hundreds to some 10000 Cherenkov photons get collected, in patterns (called the shower image), allowing to discriminate statistically those caused by primary gammas (signal) from the images of hadronic showers initiated by cosmic rays in the upper atmosphere (background).

<center><img src="img/gamma_p.png" width="600"></center>

Figure: photon (left), hadron (right)

<center><img src="img/geo.jpg" width="400"></center>

Features description:
- **Length:** continuous # major axis of ellipse [mm]
- **Width:** continuous # minor axis of ellipse [mm]
- **Size:** continuous # 10-log of sum of content of all pixels [in #phot]
- **Conc:** continuous # ratio of sum of two highest pixels over fSize [ratio]
- **Conc1:** continuous # ratio of highest pixel over fSize [ratio]
- **Asym:** continuous # distance from highest pixel to center, projected onto major axis [mm]
- **M3Long:** continuous # 3rd root of third moment along major axis [mm]
- **M3Trans:** continuous # 3rd root of third moment along minor axis [mm]
- **Alpha:** continuous # angle of major axis with vector to origin [deg]
- **Dist:** continuous # distance from origin to center of ellipse [mm]
- **Label:** g,h # gamma (signal), hadron (background)

g = gamma (signal): 12332 \
h = hadron (background): 6688

In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data

/bin/sh: wget: command not found


In [3]:
f_names = np.array(["Length", "Width", "Size", "Conc", "Conc1", "Asym", "M3Long", "M3Trans", "Alpha", "Dist"])

data = pd.read_csv("magic04.data", header=None, names=list(f_names)+["Label"])
data.head()

,Length,Width,Size,Conc,Conc1,Asym,M3Long,M3Trans,Alpha,Dist,Label
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


# Data preparation

In [4]:
# prepare a matrix of input features
X = data[f_names].values

# prepare a vector of true labels
y = 1 * (data['Label'].values == "g")

In [5]:
# scale data (apply to X, y for the further simplisity)
from sklearn.preprocessing import StandardScaler

# scale data: X = (X - mean) / sigma
X = StandardScaler().fit_transform(X)

In [6]:
X[:2]

array([[-0.57722602, -0.33680419, -0.38113037,  0.06275933, -0.14892271,
         0.54104236,  0.22481824, -0.40584194,  0.47681587, -1.49786555],
       [-0.51096889, -0.57002666, -0.64859479,  0.8203834 ,  1.471776  ,
         0.51691919,  0.26036419, -0.49009359, -0.81541816,  0.15312459]])

In [7]:
y[:5]

array([1, 1, 1, 1, 1])

# Tools

In [8]:
from sklearn import metrics

def quality_metrics_report(y_true, y_pred, y_proba):
    """
    Parameters
    ----------
    y_true: array-like of shape (n_samples,)
        Ground truth (correct) target values.
    y_pred: array-like of shape (n_samples,)
        Estimated targets as returned by a classifier.
    y_proba : array, shape = [n_samples]
        Target scores, can be probability estimates of the positive
        class.
        
    Returns
    -------
    List of metric values: [accuracy, precision, recall, f1, roc_auc]
    """
    
    accuracy  = metrics.accuracy_score(y_true, y_pred)
    precision = metrics.precision_score(y_true, y_pred)
    recall    = metrics.recall_score(y_true, y_pred)
    f1        = metrics.f1_score(y_true, y_pred)
    roc_auc   = metrics.roc_auc_score(y_true, y_proba)
    
    return [accuracy, precision, recall, f1, roc_auc]

In [9]:
from sklearn.model_selection import KFold

def kfold_uncertainties(model, X, y, n_splits=10):
    
    metrics = []
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=11)
    for train_index, test_index in kf.split(X):
        
        # fit a model
        model.fit(X[train_index], y[train_index])
        
        # make prediction
        y_test_proba = model.predict_proba(X[test_index])[:, 1]
        y_test_pred  = model.predict(X[test_index])
        
        # compute quaility metrics
        metrics_iter = quality_metrics_report(y[test_index], y_test_pred, y_test_proba)
        metrics.append(metrics_iter)
        
    metrics = np.array(metrics)
    df = pd.DataFrame()
    df['Metrics'] = columns=['Accuracy', 'Precision', 'Recall', 'F1', 'ROC AUC']
    df['Mean']    = metrics.mean(axis=0)
    df['Std']     = metrics.std(axis=0)
    
    return df

# Fit model 1

In [10]:
# import Logistic Regression classifier
from sklearn.linear_model import LogisticRegression

# define a model
model_1 = LogisticRegression(penalty='l2', C=1.0, max_iter=1000, class_weight=None, solver='lbfgs', random_state=11)

# test a model
kfold_uncertainties(model_1, X, y, n_splits=10)

,Metrics,Mean,Std
0,Accuracy,0.790694,0.006743
1,Precision,0.802153,0.010757
2,Recall,0.898820,0.008051
3,F1,0.847678,0.006583
4,ROC AUC,0.839133,0.006790


# Fit model 2

In [11]:
# import Logistic Regression classifier
from sklearn.tree import DecisionTreeClassifier

# define a model
model_2 = DecisionTreeClassifier(random_state=11)

# test a model
kfold_uncertainties(model_2, X, y, n_splits=10)

,Metrics,Mean,Std
0,Accuracy,0.817876,0.007160
1,Precision,0.860218,0.009549
2,Recall,0.858630,0.007582
3,F1,0.859379,0.006007
4,ROC AUC,0.800739,0.008184


# Resampled paired t test 

In [12]:
# install mlxtend lib
!pip install mlxtend

In [13]:
from mlxtend.evaluate import paired_ttest_resampled

t, p = paired_ttest_resampled(estimator1=model_1,
                              estimator2=model_2,
                              X=X, y=y,
                              random_seed=1, 
                              num_rounds=5)

print('t statistic: %.3f' % t)
print('p-value: %.3f' % p)

if p <= 0.05:
    print("The models are significantply different.")
else:
    print("The models are NOT significantply different.")

t statistic: -10.651
p-value: 0.000
The models are significantply different.


# Dietterich’s 5x2-Fold CV paired t test

In [14]:
from mlxtend.evaluate import paired_ttest_5x2cv

t, p = paired_ttest_5x2cv(estimator1=model_1,
                          estimator2=model_2,
                          X=X, y=y,
                          random_seed=1)

print('t statistic: %.3f' % t)
print('p-value: %.3f' % p)

if p <= 0.05:
    print("The models are significantply different.")
else:
    print("The models are NOT significantply different.")

t statistic: -6.154
p-value: 0.002
The models are significantply different.


# Alpaydin’s 5x2-Fold CV paired F test

In [15]:
from mlxtend.evaluate import combined_ftest_5x2cv

t, p = combined_ftest_5x2cv(estimator1=model_1,
                            estimator2=model_2,
                            X=X, y=y,
                            random_seed=1)

print('F statistic: %.3f' % t)
print('p-value: %.3f' % p)

if p <= 0.05:
    print("The models are significantply different.")
else:
    print("The models are NOT significantply different.")

F statistic: 22.326
p-value: 0.002
The models are significantply different.
